# Challenge 9: Solutions
## Part III: Soccer Data

*Introductory - Intermediate level SQL*

-------------------

In [1]:
from sqlalchemy import create_engine
import pandas as pd

import sqlite3
con = sqlite3.connect('/Users/mlgill/Downloads/database.sqlite')

In [2]:
pd.read_sql_query(r"""SELECT * FROM Team;""", con).head(1)

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN


In [3]:
pd.read_sql_query(r"""SELECT * FROM Match;""", con).head(1)

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.0,1.65,3.4,4.5,1.78,3.25,4.0,1.73,3.4,4.2


## Q1. Which team scored the most points when playing at home? 

In [4]:
pd.read_sql_query(r"""SELECT team_long_name, home_team_api_id, SUM(home_team_goal) AS home_goals 
                      FROM Match 
                      JOIN Team
                      ON Team.team_api_id = Match.home_team_api_id
                      GROUP BY home_team_api_id 
                      ORDER BY home_goals DESC
                      LIMIT 1;""", con)

,team_long_name,home_team_api_id,home_goals
0,Real Madrid CF,8633,505


## Q2. Did this team also score the most points when playing away? 

In [5]:
pd.read_sql_query(r"""SELECT team_long_name, away_team_api_id, SUM(away_team_goal) AS away_goals 
                      FROM Match 
                      JOIN Team 
                      ON Team.team_api_id = Match.away_team_api_id
                      GROUP BY away_team_api_id 
                      ORDER BY away_goals DESC 
                      LIMIT 1;""", con)

,team_long_name,away_team_api_id,away_goals
0,FC Barcelona,8634,354


## Q3. How many matches resulted in a tie? 

In [6]:
pd.read_sql_query(r"""SELECT COUNT(*) as num_ties
                      FROM Match
                      WHERE away_team_goal = home_team_goal;""", con)

,num_ties
0,6596


## Q4. How many players have Smith for their last name? How many have 'smith' anywhere in their name?

In [7]:
# Last name is 'Smith'
pd.read_sql_query(r"""SELECT COUNT(player_name) AS num_names
                      FROM Player
                      WHERE player_name LIKE '% Smith';""", con)

,num_names
0,15


In [8]:
# 'smith' is anywhere in the name
pd.read_sql_query(r"""SELECT COUNT(player_name) AS num_names
                      FROM Player
                      WHERE player_name LIKE '%smith%';""", con)

,num_names
0,18


## Q5. What percentage of players prefer their left or right foot? 
*Hint:* Calculate either the right or left foot, whichever is easier based on how you setup the problem.

In [9]:
pd.read_sql_query(r"""WITH _table1 AS (
                           SELECT preferred_foot, COUNT(preferred_foot) as num
                           FROM Player_Attributes
                           GROUP BY preferred_foot
                      )
                      SELECT preferred_foot, 100.0 * num / SUM(num) as percentage
                      FROM _table1""", con)

,preferred_foot,percentage
0,right,75.574691


## Q6. What was the median tie score? Use the value determined in the previous question for the number of tie games.
*Hint:* PostgreSQL does not have a median function. Instead, think about the steps required to calculate a median and use the [`WITH`](https://www.postgresql.org/docs/8.4/static/queries-with.html) command to store stepwise results as a table and then operate on these results. 

In [10]:
pd.read_sql_query(r"""WITH _table2 AS (
                           WITH _table1 AS (
                                SELECT home_team_goal
                                FROM Match
                                WHERE away_team_goal = home_team_goal
                                ORDER BY home_team_goal
                                LIMIT 6596/2 + 1
                           )
                           SELECT home_team_goal
                           FROM _table1
                           ORDER BY home_team_goal DESC
                           LIMIT 2
                      )
                      SELECT AVG(home_team_goal) AS median_tie_score
                      FROM _table2;""", con)

,median_tie_score
0,1.0


Alternative [method](https://coderwall.com/p/s53cja/calculating-median-in-postgresql) for median calculation.